In [1]:
# GRUPO
# Alexandre Galocha Pinto Junior  10734706
# Eduardo Pirro                   10734665
# Fernando Gorgulho Fayet         10734407

In [2]:
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np
import glm
import math
from PIL import Image

from lib import Object3D
from lib import Transform3D

### Código base usado em aula

In [3]:
glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
altura = 1080
largura = 1920
window = glfw.create_window(largura, altura, "T3 - CG", None, None)
glfw.make_context_current(window)


vertex_code = """
        attribute vec3 position;
        attribute vec2 texture_coord;
        attribute vec3 normals;
        
       
        varying vec2 out_texture;
        varying vec3 out_fragPos;
        varying vec3 out_normal;
                
        uniform mat4 model;
        uniform mat4 view;
        uniform mat4 projection;        
        
        void main(){
            gl_Position = projection * view * model * vec4(position,1.0);
            out_texture = vec2(texture_coord);
            out_fragPos = vec3(model * vec4(position, 1.0));
            out_normal = vec3(model * vec4(normals, 1.0));           
        }
        """

fragment_code = """

        // parametros da iluminacao ambiente e difusa
        uniform vec3 lightPos1; // define coordenadas de posicao da luz #1
        uniform vec3 lightPos2; // define coordenadas de posicao da luz #2
        uniform float ka; // coeficiente de reflexao ambiente
        uniform float kd; // coeficiente de reflexao difusa
        
        // parametros da iluminacao especular
        uniform vec3 viewPos; // define coordenadas com a posicao da camera/observador
        uniform float ks; // coeficiente de reflexao especular
        uniform float ns; // expoente de reflexao especular
        
        // parametro com a cor da(s) fonte(s) de iluminacao
        vec3 lightColor = vec3(10.0, 10.0, 10.0);

        // parametros recebidos do vertex shader
        varying vec2 out_texture; // recebido do vertex shader
        varying vec3 out_normal; // recebido do vertex shader
        varying vec3 out_fragPos; // recebido do vertex shader
        uniform sampler2D samplerTexture;
        
        
        
        void main(){
        
            // calculando reflexao ambiente
            vec3 ambient = ka * lightColor;             
        
            ////////////////////////
            // Luz #1
            ////////////////////////
            
            // calculando reflexao difusa
            // vec3 norm1 = normalize(out_normal); // normaliza vetores perpendiculares
            // vec3 lightDir1 = normalize(lightPos1 - out_fragPos); // direcao da luz
            // float diff1 = max(dot(norm1, lightDir1), 0.0); // verifica limite angular (entre 0 e 90)
            // vec3 diffuse1 = kd * diff1 * lightColor; // iluminacao difusa
            
            // calculando reflexao especular
            /* vec3 viewDir1 = normalize(viewPos - out_fragPos); // direcao do observador/camera
            vec3 reflectDir1 = reflect(-lightDir1, norm1); // direcao da reflexao
            float spec1 = pow(max(dot(viewDir1, reflectDir1), 0.0), ns);
            vec3 specular1 = ks * spec1 * lightColor;    */
            
            
            ////////////////////////
            // Luz #2
            ////////////////////////
            
            // calculando reflexao difusa
            vec3 norm2 = normalize(out_normal); // normaliza vetores perpendiculares
            vec3 lightDir2 = normalize(lightPos2 - out_fragPos); // direcao da luz
            float diff2 = max(dot(norm2, lightDir2), 0.0); // verifica limite angular (entre 0 e 90)
            vec3 diffuse2 = kd * diff2 * lightColor; // iluminacao difusa
            
            // calculando reflexao especular
            vec3 viewDir2 = normalize(viewPos - out_fragPos); // direcao do observador/camera
            vec3 reflectDir2 = reflect(-lightDir2, norm2); // direcao da reflexao
            float spec2 = pow(max(dot(viewDir2, reflectDir2), 0.0), ns);
            vec3 specular2 = ks * spec2 * lightColor;    
            
            ////////////////////////
            // Combinando as duas fontes
            ////////////////////////
            
            // aplicando o modelo de iluminacao
            vec4 texture = texture2D(samplerTexture, out_texture);
            vec4 result = vec4((ambient + diffuse2 + specular2),1.0) * texture; // aplica iluminacao
            gl_FragColor = result;

        }
        """

# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)

# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")
    
glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")
    
# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)

# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)

### Função de carregamento de arquivos de textura

In [4]:
# glHint(GL_LINE_SMOOTH_HINT, GL_DONT_CARE)
# glEnable( GL_BLEND )
# glBlendFunc( GL_SRC_ALPHA, GL_ONE_MINUS_SRC_ALPHA )
# glEnable(GL_LINE_SMOOTH)
glEnable(GL_TEXTURE_2D)
qtd_texturas = 15
textures = glGenTextures(qtd_texturas)

def load_texture_from_file(texture_id, img_textura):
    glBindTexture(GL_TEXTURE_2D, texture_id)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_REPEAT)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR)
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR)
    img = Image.open(img_textura)
    img_width = img.size[0]
    img_height = img.size[1]
    image_data = img.tobytes("raw", "RGB", 0, -1)
#     image_data = img.convert("RGBA").tobytes("raw", "RGBA",0,-1)

#     glTexImage2D(GL_TEXTURE_2D, 0, GL_RGBA, img_width, img_height, 0, GL_RGBA, GL_UNSIGNED_BYTE, image_data)
    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, img_width, img_height, 0, GL_RGB, GL_UNSIGNED_BYTE, image_data)

### Criação dos modelos

As texturas referentes a cada modelo devem ser carregadas no código principal com a função 'load_texture_from_file'

Para cada modelo, cria-se um object3d e carrega-se os vértices através da função 'load_from_file'

Para cada modelo, deve-se usar a função 'bind_textures' para indicar o id das texturas usadas pelo modelo

In [5]:
### Chão de pedra
# load_texture_from_file(0, 'models/stone_floor/txt_stone_floor.jpg')

# stone_floor = Object3D.Object3D()
# stone_floor.t_y = -1.01
# stone_floor.s_x = 10.0
# stone_floor.s_y = 10.0
# stone_floor.s_z = 10.0

# stone_floor.load_from_file('models/stone_floor/mdl_stone_floor.obj')
# stone_floor.bind_textures([0])

In [6]:
### Casa
load_texture_from_file(3, 'models/house/txt_house_1.jpg')
load_texture_from_file(2, 'models/house/txt_house_2.png')
load_texture_from_file(1, 'models/house/txt_house_3.png')

house = Object3D.Object3D()
house.t_y = -1.0
house.s_x = 0.02
house.s_y = 0.02
house.s_z = 0.02

house.load_from_file('models/house/mdl_house.obj')
house.bind_textures([1,2,3])

In [7]:
### Mesa (dentro da casa)
load_texture_from_file(4, 'models/table/txt_table.png')

table = Object3D.Object3D()
table.t_y = -1.0

table.load_from_file('models/table/mdl_table.obj')
table.bind_textures([4,4])

In [8]:
### Homem Aranha
load_texture_from_file(5, 'models/spiderman/txt_spiderman.png')

spiderman = Object3D.Object3D()
spiderman.t_y = -1.0
spiderman.t_z = -1.1
spiderman.t_x = -1.3
spiderman.a_y = 55
spiderman.s_x = 0.15
spiderman.s_y = 0.15
spiderman.s_z = 0.15

spiderman.load_from_file('models/spiderman/mdl_spiderman.obj')
spiderman.bind_textures([5])

In [9]:
### Árvore
load_texture_from_file(6, 'models/tree/txt_tree_1.jpg')
load_texture_from_file(7, 'models/tree/txt_tree_2.png')

tree = Object3D.Object3D()
tree.t_y = -1.0
tree.t_z = -3.1
tree.t_x = +7.0
tree.s_x = 0.75
tree.s_y = 0.75
tree.s_z = 0.75

tree.load_from_file('models/tree/mdl_tree.obj')
tree.bind_textures([6,7])

In [10]:
### Tanques
load_texture_from_file(8, 'models/tanks/txt_tanks.jpg')

tanks = Object3D.Object3D()
tanks.t_y = -1.0
tanks.t_z = -7
tanks.t_x = -7.0
tanks.a_x = -90
tanks.s_x = 0.5
tanks.s_y = 0.5
tanks.s_z = 0.5

tanks.load_from_file('models/tanks/mdl_tanks.obj')
tanks.bind_textures([8,8,8,8,8])

In [11]:
### Avião
load_texture_from_file(9, 'models/plane/txt_plane_1.jpg')
load_texture_from_file(10, 'models/plane/txt_plane_2.jpg')

plane = Object3D.Object3D()
plane.t_y = 10.0
plane.t_x = -20.0
plane.a_x = -90
plane.a_y = +30
plane.s_x = 0.001
plane.s_y = 0.001
plane.s_z = 0.001

plane.load_from_file('models/plane/mdl_plane.obj')
plane.bind_textures([9,10])

In [12]:
### Sky box
load_texture_from_file(11, 'models/sky/txt_sky.png')

sky_box = Object3D.Object3D()
sky_box.s_x = 2.0
sky_box.s_y = 2.0
sky_box.s_z = 2.0

sky_box.load_from_file('models/sky/mdl_sky.obj')
sky_box.bind_textures([11])

In [13]:
### Notebook
load_texture_from_file(13, 'models/notebook/txt_notebook_1.jpg')
load_texture_from_file(12, 'models/notebook/txt_notebook_2.jpg')

notebook = Object3D.Object3D()
notebook.t_y = -0.64
notebook.s_x = 0.1
notebook.s_y = 0.1
notebook.s_z = 0.1

notebook.load_from_file('models/notebook/mdl_notebook.obj')
notebook.bind_textures([12, 13])

In [14]:
### Notebook2

notebook2 = Object3D.Object3D()
notebook2.s_x = 0.1
notebook2.s_y = 0.1
notebook2.s_z = 0.1
notebook2.t_x = 28
notebook2.t_y = 30
notebook2.t_z = 30

notebook2.load_from_file('models/notebook/mdl_notebook.obj')
notebook2.bind_textures([12, 13])

In [15]:
### Chão de grama
# load_texture_from_file(14, 'models/grass/txt_grass.jpg')

# grass = Object3D.Object3D()
# grass.t_y = -1.011
# grass.s_x = 100.0
# grass.s_y = 100.0
# grass.s_z = 100.0

# grass.load_from_file('models/grass/mdl_grass.obj')
# grass.bind_textures([14])

In [16]:
# all_objects = [stone_floor, grass, sky_box, house, table, spiderman, tree, tanks, plane, notebook]
all_objects = [sky_box, house, table, spiderman, tree, tanks, plane, notebook, notebook2]

### Preparando os dados e enviando para a GPU

In [17]:
# Request a buffer slot from GPU
buffer = glGenBuffers(3)

serialized_vertices, serialized_tex_coords, serialized_normals = Object3D.serialize_objects3d(all_objects)



vertices = np.zeros(len(serialized_vertices), [("position", np.float32, 3)])
vertices['position'] = serialized_vertices

# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[0])
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_STATIC_DRAW)
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)
loc_vertices = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc_vertices)
glVertexAttribPointer(loc_vertices, 3, GL_FLOAT, False, stride, offset)



textures = np.zeros(len(serialized_tex_coords), [("position", np.float32, 2)]) # duas coordenadas
textures['position'] = serialized_tex_coords

# Upload data
glBindBuffer(GL_ARRAY_BUFFER, buffer[1])
glBufferData(GL_ARRAY_BUFFER, textures.nbytes, textures, GL_STATIC_DRAW)
stride = textures.strides[0]
offset = ctypes.c_void_p(0)
loc_texture_coord = glGetAttribLocation(program, "texture_coord")
glEnableVertexAttribArray(loc_texture_coord)
glVertexAttribPointer(loc_texture_coord, 2, GL_FLOAT, False, stride, offset)



normals = np.zeros(len(serialized_normals), [("position", np.float32, 3)]) # três coordenadas
normals['position'] = serialized_normals

# Upload coordenadas normals de cada vertice
glBindBuffer(GL_ARRAY_BUFFER, buffer[2])
glBufferData(GL_ARRAY_BUFFER, normals.nbytes, normals, GL_STATIC_DRAW)
stride = normals.strides[0]
offset = ctypes.c_void_p(0)
loc_normals_coord = glGetAttribLocation(program, "normals")
glEnableVertexAttribArray(loc_normals_coord)
glVertexAttribPointer(loc_normals_coord, 3, GL_FLOAT, False, stride, offset)

### Eventos para modificar a posição da câmera.

* Movimentação com as teclas A, S, D e W
* Foco da câmera com o mouse
* A janela captura o mouse! Para fecha-la, precione Alt-F4

In [18]:
cameraPos   = glm.vec3(-1.0,  -0.2,  0.7);
cameraFront = glm.vec3(0.0,  0.0, -1.0);
cameraUp    = glm.vec3(0.0,  1.0,  0.0);

skybox_radius = 50                    # raio a partir da origem em que o jogador pode se movimentar

polygonal_mode = False

def key_event(window,key,scancode,action,mods):
    global cameraPos, cameraFront, cameraUp, polygonal_mode
    
    cameraSpeed = 0.2
    if key == 87 and (action==1 or action==2): # tecla W
        newCamPos = cameraPos + cameraSpeed * cameraFront
        if(glm.length(newCamPos) < skybox_radius and newCamPos[1] > -0.9):
            cameraPos = newCamPos
    
    if key == 83 and (action==1 or action==2): # tecla S
        newCamPos = cameraPos - cameraSpeed * cameraFront
        if(glm.length(newCamPos) < skybox_radius and newCamPos[1] > -0.9):
            cameraPos = newCamPos
    
    if key == 65 and (action==1 or action==2): # tecla A
        newCamPos = cameraPos - glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        if(glm.length(newCamPos) < skybox_radius and newCamPos[1] > -0.9):
            cameraPos = newCamPos
        
    if key == 68 and (action==1 or action==2): # tecla D
        newCamPos = cameraPos + glm.normalize(glm.cross(cameraFront, cameraUp)) * cameraSpeed
        if(glm.length(newCamPos) < skybox_radius and newCamPos[1] > -0.9):
            cameraPos = newCamPos
        
    if key == 80 and action==1 and polygonal_mode==True:
        polygonal_mode=False
    else:
        if key == 80 and action==1 and polygonal_mode==False:
            polygonal_mode=True
            
        
        
        
firstMouse = True
yaw = -90.0 
pitch = 0.0
lastX =  largura/2
lastY =  altura/2

def mouse_event(window, xpos, ypos):
    global firstMouse, cameraFront, yaw, pitch, lastX, lastY
    if firstMouse:
        lastX = xpos
        lastY = ypos
        firstMouse = False

    xoffset = xpos - lastX
    yoffset = lastY - ypos
    lastX = xpos
    lastY = ypos

    sensitivity = 0.1 
    xoffset *= sensitivity
    yoffset *= sensitivity

    yaw += xoffset;
    pitch += yoffset;

    
    if pitch >= 90.0: pitch = 90.0
    if pitch <= -90.0: pitch = -90.0

    front = glm.vec3()
    front.x = math.cos(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    front.y = math.sin(glm.radians(pitch))
    front.z = math.sin(glm.radians(yaw)) * math.cos(glm.radians(pitch))
    cameraFront = glm.normalize(front)


glfw.set_input_mode(window, glfw.CURSOR, glfw.CURSOR_DISABLED);     # captura do mouse pela janela
glfw.set_key_callback(window,key_event)
glfw.set_cursor_pos_callback(window, mouse_event)

### Matrizes View e Projection

In [19]:
def view():
    global cameraPos, cameraFront, cameraUp
    mat_view = glm.lookAt(cameraPos, cameraPos + cameraFront, cameraUp);
    mat_view = np.array(mat_view)
    return mat_view

def projection():
    global altura, largura
    # perspective parameters: fovy, aspect, near, far
    mat_projection = glm.perspective(glm.radians(45.0), largura/altura, 0.1, 1000.0)
    mat_projection = np.array(mat_projection)    
    return mat_projection

### Loop principal
* Lógica para a rotação do avião
* Desenho dos objetos
* Controles: ASWD e mouse
* USE Alt-F4 PARA FECHAR A JANELA

In [20]:
glfw.show_window(window)
glfw.set_cursor_pos(window, lastX, lastY)

glEnable(GL_DEPTH_TEST)

# controle para a rotação do avião
plane_angle = 0


# loc_light_pos = glGetUniformLocation(program, "lightPos1") # recuperando localizacao da variavel lightPos na GPU
# glUniform3f(loc_light_pos, 0, 0, 0)


loc_light_pos = glGetUniformLocation(program, "lightPos2") # recuperando localizacao da variavel lightPos na GPU
glUniform3f(loc_light_pos, 30, 30, 30)
   

while not glfw.window_should_close(window):

    glfw.poll_events() 
    
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    
    glClearColor(1.0, 1.0, 1.0, 1.0)
    
    if polygonal_mode==True:
        glPolygonMode(GL_FRONT_AND_BACK,GL_LINE)
    if polygonal_mode==False:
        glPolygonMode(GL_FRONT_AND_BACK,GL_FILL)
        
        
        
    # Lógica do avião. A rotação é constantemente decrementada
    # Uma matriz adicional de rotação é gerada e utilizada no passo
    # de desenho do avião
    plane_angle -= 0.1
    plane_extra_mat = Transform3D.rotacao(plane_angle, (0, 0, 0), 'y')
    
    
    
    # Desenho dos objetos. Para o avião especificamente, uma matriz extra de rotação 
    # é computada para o desenho
    loc_model = glGetUniformLocation(program, "model")
    
    offset = 0
    for obj in all_objects:
        if(obj == plane):
            offset = obj.render(program, loc_model, offset, [plane_extra_mat])
        else:
            offset = obj.render(program, loc_model, offset)

    
    
    
    mat_view = view()
    loc_view = glGetUniformLocation(program, "view")
    glUniformMatrix4fv(loc_view, 1, GL_FALSE, mat_view)

    mat_projection = projection()
    loc_projection = glGetUniformLocation(program, "projection")
    glUniformMatrix4fv(loc_projection, 1, GL_FALSE, mat_projection)    
    
    glfw.swap_buffers(window)

glfw.terminate()